In [2]:
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = "/content/drive/MyDrive/datathon/data/"

SEED = 42

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:
train_tr = pd.read_csv(f"{DATA_PATH}train_1226.csv") # 학습용 승부예측 데이터
train_target = pd.read_csv(f"{DATA_PATH}train_target_1226.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}test_1226.csv") # 테스트용 승부예측 데이터
test_target = pd.read_csv(f"{DATA_PATH}test_target_1226.csv") # 테스트용 정답 데이터

In [20]:
train_tr.shape, train_target.shape, test_tr.shape, test_target.shape

((266040, 18), (266040, 1), (67380, 18), (67380, 1))

# 년월일시 분리

In [22]:
train_tr["date"] = pd.to_datetime(train_tr["date"])
test_tr["date"] = pd.to_datetime(test_tr["date"])

train_tr["year"] = train_tr["date"].dt.year
train_tr["month"] = train_tr["date"].dt.month
train_tr["day"] = train_tr["date"].dt.day
train_tr["time"] = train_tr["date"].dt.hour

test_tr["year"] = test_tr["date"].dt.year
test_tr["month"] = test_tr["date"].dt.month
test_tr["day"] = test_tr["date"].dt.day
test_tr["time"] = test_tr["date"].dt.hour

train_tr.drop(columns=["date"], inplace=True)
test_tr.drop(columns=["date"], inplace=True)

In [25]:
train_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266040 entries, 0 to 266039
Data columns (total 20 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   league    266040 non-null  object 
 1   year      266040 non-null  int32  
 2   patch     266040 non-null  float64
 3   side      266040 non-null  object 
 4   position  266040 non-null  object 
 5   playerid  218803 non-null  object 
 6   teamid    262776 non-null  object 
 7   ban1      265248 non-null  object 
 8   ban2      265602 non-null  object 
 9   ban3      265002 non-null  object 
 10  ban4      265254 non-null  object 
 11  ban5      264606 non-null  object 
 12  pick1     36686 non-null   object 
 13  pick2     36686 non-null   object 
 14  pick3     36686 non-null   object 
 15  pick4     36686 non-null   object 
 16  pick5     36686 non-null   object 
 17  month     266040 non-null  int32  
 18  day       266040 non-null  int32  
 19  time      266040 non-null  int32  
dtypes: f

In [24]:

# 시간대별 데이터 확인 (train_tr)
print(train_tr.groupby('time').size())

# 시간대별 데이터 확인 (test_tr)
print(test_tr.groupby('time').size())

time
0     12144
1      9996
2      5508
3      2592
4      2628
5      5100
6      4992
7      5652
8      9156
9     10488
10    10980
11     9816
12     7560
13     5304
14     5280
15    10080
16    18792
17    26220
18    24768
19    22668
20    17184
21    13236
22    12600
23    13296
dtype: int64
time
0     2364
1     1452
2      756
3      420
4       36
5     2268
6     1956
7     1620
8     2364
9     2016
10    2616
11    2184
12    2148
13    1284
14     924
15    3912
16    8112
17    9084
18    7992
19    6000
20    3024
21    1896
22    1488
23    1464
dtype: int64


# 피처

- 시간대별(경기) : 였는데 시간대가 안맞아 뺌

In [39]:

# teamid가 oe:team:78f183fa5a7d8ecb22b9ad272c3abd7인 팀의 2023년 2월 경기 시간 출력
print(train_tr[(train_tr["teamid"] == "oe:team:78f183fa5a7d8ecb22b9ad272c3abd7") & (train_tr["year"] == 2023) & (train_tr["month"] == 2)][["year", "month", "day", "time"]])

        year  month  day  time
136932  2023      2    4    19
136933  2023      2    4    19
136934  2023      2    4    19
136935  2023      2    4    19
136936  2023      2    4    19
136942  2023      2    4    19
137669  2023      2    5    20
137670  2023      2    5    20
137671  2023      2    5    20
137672  2023      2    5    20
137673  2023      2    5    20
137675  2023      2    5    20
138101  2023      2    6    19
138102  2023      2    6    19
138103  2023      2    6    19
138104  2023      2    6    19
138105  2023      2    6    19
138107  2023      2    6    19
